In [1]:
# import os
import numpy as np
import tensorflow as tf
# import pickle as pkl
import json
from mqtt_sub import sub_iot_platform_cin_con
from MobiusAPI import http_post_get

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
# 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8000)])
    except RuntimeError as e:
        # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

2023-10-17 05:41:37.410095: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 05:41:37.570733: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-17 05:41:37.603070: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-17 05:41:38.246374: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
model = tf.keras.models.load_model('model/resnet_cylinder')

nu = "mqtt://203.250.148.120/subscriptionRawData?ct=json"
Port = 20516

URI = '/Mobius/radarEye/report/countingCylinderType'

AE_ID = 'radarEye1'

2023-10-17 05:41:39.292225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 05:41:39.991781: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8000 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6


In [3]:
while True:
    json_data = sub_iot_platform_cin_con(nu, Port)
    print(type(json_data['data_1']))
    data = np.array(json_data['data_1'])
    print(data.shape)
    y_pred = model.predict(data)
    # print(y_pred)
    y = y_pred[0]
    pred = dict()
    for i in range(len(y)):
        pred[str(i).zfill(2)] = y[i].tolist()

    send_data = json.dumps(pred)
    http_post_get.mobius_post(URI, AE_ID, AE_ID, send_data)

mqtt://203.250.148.120/subscriptionRawData?ct=json
subscriptionRawData
connected OK
subscribed: 1 (1,)
{'op': 5, 'rqi': 'ghwFttILBvl', 'to': 'mqtt://203.250.148.120/subscriptionRawData?ct=json', 'fr': '/Mobius2', 'pc': {'m2m:sgn': {'sur': 'Mobius/radarEye/sensor1/rawdata/subscriptionRawData', 'nev': {'rep': {'m2m:cin': {'rn': '4-20231017054143796', 'ty': 4, 'pi': '3-20231017053609095349', 'ri': '4-20231017054143798642', 'ct': '20231017T054143', 'lt': '20231017T054143', 'st': 2, 'et': '20251017T054143', 'cs': 6130, 'con': {'data_1': [[[20.219297712828705], [37.22002149381432], [12.614673994994877], [7.349829930005185], [6.382005954243541], [21.073680267100954], [36.30895206419486], [23.39615353001429], [74.51831989517746], [27.122315535366816], [31.470780098370618], [62.98587143161552], [6.216912416947822], [55.19003533247646], [31.928827100286664], [59.175163709110265], [37.70225457449462], [63.78095326976542], [29.165047574108293], [7.924645102463581], [27.780028797681258], [34.149231

2023-10-17 05:41:45.272476: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-17 05:41:45.458006: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


1/1 [==============================] - 3s 3s/step
{'Accept': 'application/json', 'X-M2M-RI': 'radarEye1', 'X-M2M-Origin': 'radarEye1', 'Content-Type': 'application/vnd.onem2m-res+json ; ty = 4'}
{"m2m:cin":{"rn":"4-20231017054147158","ty":4,"pi":"3-20231017053656142534","ri":"4-20231017054147159097","ct":"20231017T054147","lt":"20231017T054147","st":1,"et":"20251017T054147","cs":467,"con":{"10":0.000001206155729960301,"11":0.000013710950042877812,"12":0.0016887201927602291,"13":0.0000028826289053540677,"14":0.000007625232683494687,"15":0.000035853401641361415,"16":4.649393048339334e-8,"00":1.7641332838991275e-10,"01":0.00016800056619103998,"02":0.9879062175750732,"03":0.002372003858909011,"04":0.00002648885674716439,"05":0.0004233355284668505,"06":0.0021333887707442045,"07":0.004399128723889589,"08":1.482521128082226e-7,"09":0.0008212574175558984},"cr":"radarEye1"}}
mqtt://203.250.148.120/subscriptionRawData?ct=json
subscriptionRawData
connected OK
subscribed: 1 (1,)
{'op': 5, 'rqi': '